## LinkedIn Profile Scrapper in Python

In [1]:
import requests, time, random
from bs4 import BeautifulSoup 
from selenium import webdriver
import pandas as pd
import operator
import pyodbc
import requests
import time
from datetime import datetime

In [2]:
#ejemplo
#VARIABLES
#VARIABLES DE CONFIGURACION INICIAL
SESION_PC = 'ALEX'
TABLA_BUSCAR = 'REV_PARQUE'
TABLA_BUSCAR2 = 'mae_titular_uba'

TABLA_INSERTAR = 'AV_BASE_LINKEDIN'
SERVER = 'SVRBI'
DB = 'BDANALYTICS'
HOY = datetime.now().strftime('%Y-%m-%d')
linkRef = []
#file = open('config.txt') 
#datos = file.readlines()
#datos

In [3]:
def slideBar(num, browser):
    SCROLL_PAUSE_TIME = 10

    # Get scroll height
    last_height = browser.execute_script("return document.body.scrollHeight")

    for i in range(num):
        # Scroll down to bottom
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight/2);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = browser.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

In [4]:
def iniciarSesion(browser):
    browser.get('https://www.linkedin.com/uas/login')
    file = open('config.txt')
    lines = file.readlines()
    username = lines[0]
    password = lines[1]


    elementID = browser.find_element_by_id('username')
    elementID.send_keys(username)

    elementID = browser.find_element_by_id('password')
    elementID.send_keys(password)
    elementID.submit()

In [5]:
# Exact Link Text
def busquedaPersonalizada(linkRef, name, browser, estado):
    linkRef = []
    if estado == 0:
        browser.get('https://www.linkedin.com/mwlite/search/results/all?keywords='+ name[2]+'+'+name[0]+'+'+name[1])
    else:
        browser.get('https://www.linkedin.com/mwlite/search/results/all?keywords='+ name[2]+'+'+name[0])
        
    #slideBar(1, browser)
    
    #obtengo el link personal de cada perfil que se encuentre
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    if soup.find('ul', {'class': 'search-results'}) != None:
        name_div = soup.find('ul', {'class': 'search-results'})
        div = name_div.find_all('div',{'class': 'flex-entity-lockup'})

        for div1 in div:
            dataLink = div1.find_all('a',{ 'class': 'profile-path primary-details'} , href = True)
            for a in dataLink:
                linkRef.append(a['href']) 
                
    return linkRef
        

In [6]:

def main(linkRef, browser):
    path = 'https://www.linkedin.com'
    browser.get(path+linkRef) 
    
    slideBar(4, browser)
    
    src = browser.page_source
    soup = BeautifulSoup(src, 'lxml')
    
    info = []
#variables
    name = ""
    loc = ""
    profile_title = ""
    company_name = ""
    joining_date = ""
    college_name = ""
    carreer = ""
    job_title = ""
    
#datosPersonales
    name_div = soup.find('div', {'class': 'flex-1 mr5'})
    name_loc = name_div.find_all('ul')
    name = name_loc[0].find('li').get_text().strip()
    loc = name_loc[1].find('li').get_text().strip()
    profile_title = name_div.find('h2').get_text().strip()
    
#experiencia
#####trabajar esta parte:
    try:
        exp_section = soup.find('section', {'id': 'experience-section'}).find('ul')
        li_tags = exp_section.find('div')
        a_tags = li_tags.find('a')
        job_title = a_tags.find('h3').get_text().strip()
        company_name = a_tags.find_all('p')[1].get_text().strip()
        #joining_date = a_tags.find_all('h4')[0].find_all('span')[1].get_text().strip()
    except Exception:
        pass

#educacion
    try:
        edu_section = soup.find('section', {'id': 'education-section'}).find('ul')     
        college_name = edu_section.find('h3').get_text().strip()
        #carreer = edu_section.find('p', {'class': 'pv-entity__secondary-title pv-entity__fos t-14 t-black t-normal'}).find_all('span')[1].get_text().strip()
    except Exception:
        pass
    
    
    info.append(name)
    info.append(loc) 
    info.append(profile_title)
    info.append(company_name) 
    info.append(job_title) 
    info.append(college_name) 
    #info.append(carreer) 
    
    return info

In [147]:
###pruebas ad
 
src = browser.page_source

soup = BeautifulSoup(src, 'lxml')

#divP = soup.find('div',{'class':'profile-detail'})
exp_section = soup.find('section', {'id': 'experience-section'}).find('ul')
li_tags = exp_section.find('div')
a_tags = li_tags.find('a')
job_title = a_tags.find('h3').get_text().strip()
company_name = a_tags.find_all('p')[1].get_text().strip()
 #joining_date = a_tags.find_all('h4')[0].find_all('span')[1].get_text().strip()
edu_section = soup.find('section', {'id': 'education-section'}).find('ul')     
college_name = edu_section.find('h3').get_text().strip()
carreer = edu_section.find('p', {'class': 'pv-entity__secondary-title pv-entity__fos t-14 t-black t-normal'}).find_all('span')[1].get_text().strip()

print(job_title, company_name, college_name)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.117)


In [7]:
def baseData():
    cnn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DB)
    cursor = cnn.cursor()

    cursor.execute("select top 100 b.numdoc ,b.nombre_largo from "+TABLA_BUSCAR+" a left join "+TABLA_BUSCAR2 +" b on a.ID_cuenta = b.idencuenta WHERE PERIODO='202001' ")

    records = cursor.fetchall()
    cursor.close()
    return records

In [8]:
def tablaInsertar(i, info, estado):
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+SERVER+';DATABASE='+DB)
    cursor = conn.cursor()
    conn.cursor().execute("insert into AV_BASE_LINKEDIN values (?,?,?,?,?,?,?,?)",(i[0],info[0],info[1], info[3], info[2], info[5], estado,HOY ) )
    conn.commit()
    conn.close()

In [9]:
if __name__ == '__main__':
   
    records = baseData()
    browser = webdriver.Chrome('chromedriver.exe')
    iniciarSesion(browser)         
    
    for i in records:
        name = i[1].split()
        estado = 0
        linkRef = busquedaPersonalizada(linkRef, name, browser,estado) 
        cont = 0
        print (i)
        
        if len(linkRef) == 0:
            estado = 1
            linkRef = busquedaPersonalizada(linkRef, name, browser,estado)    
        #info de cada perfil    
        for j in linkRef:            
            info = main(j,browser) 
            
            if cont == 4:
                break
                ##para extraer solo aquellos que son de Perú
            if info[1].find('erú') == 1 or info[1].find('eru') == 1:
                tablaInsertar(i, info)   
                print(info)
                break;
            cont+=1
            

('42869811', 'BRAVO ALCCA RICHARD JOHN')
['Richard Alberto Lara Bravo', 'Perú', 'Administrativo Multifuncional en BBVA Provincial', 'BBVA Provincial', 'Administrativo Multifuncional', 'Universidad Yacambú']
('07809608', 'CASAPIA VALDIVIA GUILLERMO HUMBERTO')
['Guillermo Casapía', 'Perú', 'Manager HEAT COMPANY S.A.C.', '', '', '']
('10090213', 'CENTENO BARRIENTOS JUAN DANY')
['juan dany centeno barrientos', 'Perú', '--', '', '', '']
('10307110', 'PEREZ MEDINA LITTA PATRICIA')
('44970275', 'TANIGUCHI PAREDES CYNTHIA JANINA')
['Cynthia Taniguchi', 'Perú', 'Analista de QA en TBS Peru', 'TBS Peru', 'Analista de QA', '']
('42154417', 'MENDOZA URRUTIA ANGEL IVAN')


DataError: ('22001', '[22001] [Microsoft][ODBC SQL Server Driver][SQL Server]Los datos de cadena o binarios se truncarían. (8152) (SQLExecDirectW); [22001] [Microsoft][ODBC SQL Server Driver][SQL Server]Se terminó la instrucción. (3621)')